### `Determines substitution effects of all significant markers`

In [39]:
import numpy as np
import pandas as pd
import researchpy as rp
import scipy.stats as stats
from scipy.stats import shapiro
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna

In [ ]:
#make candidate genes table

In [1157]:
df = pd.read_excel('../Figures and tables/Candidate genes.xlsx')

In [1158]:
df2 = pd.read_csv('../Figures and tables/Expression dendogram 10 clusters.csv')

In [1159]:
df2 = pd.merge(df,df2, how = 'right').fillna('No')

In [1160]:
df3 = pd.read_table('../../Data/MSU7/all.locus_brief_info.7.0').drop_duplicates(['locus', 'annotation'])[['locus', 'annotation']]

In [1161]:
df3 = pd.merge(df2, df3, left_on = 'Locus ID', right_on = 'locus')

In [1068]:
#write table
df3[['Locus ID', 'QTL', 'annotation', 'Functionally annotated', 'Outlier in expression', 'Expression cluster',
    'Special interest']].to_csv('../Figures and tables/All candidate genes.csv', index = False)

In [1070]:
#make fasta
for sr in SeqIO.parse('../../Data/MSU7/all.seq', 'fasta'):
    if sr.id in df3['Locus ID'].to_list():
        with open('../Sequence and SNP data/340 candidate genes.fasta', 'a') as f:
            f.write('>%s\n%s\n' %(sr.description, sr.seq))

In [1072]:
#test fasta
c = 0
for sr in SeqIO.parse('../Sequence and SNP data/340 candidate genes.fasta', 'fasta'):
    c+=1
print(c)

340


In [ ]:
#gonna go over to run blast and return with blast output

In [1162]:
df4 = pd.read_table('../Sequence and SNP data/Bout340can.tsv', header = None)
df4['Start'] = df4[[8,9]].min(axis = 1)
df4['End'] = df4[[8,9]].max(axis = 1)
t = []
for s,e in df4[[8,9]].itertuples(index = False):
    if s < e:
        t.append('forward')
    else:
        t.append('reverse')
df4['Strand'] = t

In [1163]:
df4[df4[0].duplicated(keep = False)]

,0,1,2,3,4,5,6,7,8,9,10,11,Start,End,Strand
47,LOC_Os02g15178,chr02,100.000,2139,0,0,1,2139,8460664,8458526,0.000000e+00,3951,8458526,8460664,reverse
48,LOC_Os02g15178,chr02,100.000,2139,0,0,1,2139,8465035,8467173,0.000000e+00,3951,8465035,8467173,forward
122,LOC_Os05g30410,chr05,100.000,333,0,0,1,333,17627376,17627044,4.060000e-175,616,17627044,17627376,reverse
123,LOC_Os05g30410,chr04,93.994,333,20,0,1,333,33855052,33854720,9.160000e-142,505,33854720,33855052,reverse
144,LOC_Os06g35574,chr06,100.000,898,0,0,1,898,20761602,20760705,0.000000e+00,1659,20760705,20761602,reverse
145,LOC_Os06g35574,chr03,85.761,927,88,31,1,898,31959677,31960588,0.000000e+00,941,31959677,31960588,forward
206,LOC_Os09g15350,chr09,100.000,2370,0,0,1,2370,9385267,9387636,0.000000e+00,4377,9385267,9387636,forward
207,LOC_Os09g15350,chr05,98.819,2370,13,2,1,2370,17778572,17776218,0.000000e+00,4207,17776218,17778572,reverse
318,LOC_Os12g28137,chr12,100.000,1987,0,0,1,1987,16616739,16618725,0.000000e+00,3670,16616739,16618725,forward
319,LOC_Os12g28137,chr12,100.000,1987,0,0,1,1987,16640708,16642694,0.000000e+00,3670,16640708,16642694,forward


In [1164]:
#fix nonlocus hits
df5 = df4.loc[[123, 145, 207],:].copy()
df5.to_csv('../Figures and tables/Nonloci Bhits.csv', index = False)

In [ ]:
#get marker range

In [1165]:
df4 = df4.drop(index = [123,145,207])

In [1166]:
df4 = pd.merge(df4, df3[['Locus ID', 'QTL']], left_on = 0, right_on = 'Locus ID')[[0, 1, 'Start', 'End', 'Strand', 'QTL']]

In [1167]:
df5 = pd.concat([df4.groupby('QTL').min()['Start'], df4.groupby('QTL').max()['End']], axis = 1)

In [1168]:
df5 = pd.merge(df5.reset_index(), df4[['QTL', 1]].drop_duplicates())

In [1115]:
df5.to_csv('../Sequence and SNP data/340can range.csv', index = False)

In [ ]:
#Filter sliced VCF file

In [1169]:
df5 = pd.read_csv('../../WFC2.vcf', sep = '\t')[['#CHROM', 'POS', 'ID', 'REF', 'ALT']]

In [1170]:
t = []
for c, p in df5[['#CHROM', 'POS']].itertuples(index = False):
    f = False
    for l,o,s,e in df4[df4[1] == c][[0, 'Strand', 'Start', 'End']].itertuples(index = False):
        if s <= p <= e:
            t.append((l, o, s, e))
            f = True
            break
    if f == False:
        t.append(('None', o, s, e))

In [1171]:
df5['Gene'] = list(zip(*t))[0]
df5['Strand'] = list(zip(*t))[1]
df5['Start'] = list(zip(*t))[2]
df5['End'] = list(zip(*t))[3]

In [1172]:
df5 = df5[df5['Gene'] != 'None']

In [187]:
#don't run
ds = []
for sr in SeqIO.parse('../../Data/RAP-DB/IRGSP-1.0_genome.fasta', 'fasta'):
    tdf = df5[df5['#CHROM'] == sr.id]
    tdf = tdf.drop_duplicates(['Gene', 'Start', 'End'])
    for r,p,g,s,s2,e in tdf[['REF', 'POS', 'Gene', 'Strand', 'Start', 'End']].itertuples(index = False):
        assert sr.seq[p-1].lower() == r[0].lower()        
        if s == 'forward':
            ds.append(SeqRecord(sr.seq[(s2-1):e], id = g + '_' + str(s2)))
        else:
            ds.append(SeqRecord(sr.seq[(s2-1):e].reverse_complement(), id = g + '_' + str(s2)))
with open('extracted_candidates.fasta', 'w') as f:
    SeqIO.write(ds, f, 'fasta')

In [1134]:
#test
c = 0
for sr in SeqIO.parse('../Sequence and SNP data/340 candidate genes.fasta', 'fasta'):
    if sr.id not in df4[0].to_list():
        print(sr.id)
    c+=1
print(c)

340


In [ ]:
#annotate blast hits

In [1249]:
df6 = pd.read_table('../../Data/MSU7/all.locus_brief_info.7.0')

In [1250]:
df6 = df6[df6['is_expressed'] == 'Y']

In [1186]:
df6r = df6[df6['is_representative'] == 'Y']

In [1188]:
df6nr = df6[df6['is_representative'] == 'N']

In [1189]:
#non representative hits
for l in df6nr['locus'][df6nr['locus'].duplicated()].to_list():
    if l in df4[0].to_list():
        print(l)

LOC_Os03g03470
LOC_Os03g03470
LOC_Os03g03610
LOC_Os03g29350
LOC_Os05g30280
LOC_Os05g30400
LOC_Os05g30400
LOC_Os07g36600
LOC_Os07g37580
LOC_Os08g43360
LOC_Os09g15430
LOC_Os09g15430
LOC_Os09g34280
LOC_Os09g34280
LOC_Os09g34310
LOC_Os09g34850
LOC_Os10g35030
LOC_Os10g35070
LOC_Os10g35110
LOC_Os12g41650
LOC_Os12g41650
LOC_Os12g41710
LOC_Os12g41710


In [1292]:
df7 = pd.merge(df4,df6[['locus', 'model']], left_on = 0, right_on = 'locus')

In [1293]:
df7 = df7.drop_duplicates(['locus', 'model'])

In [1227]:
#check if all models are present
t = []
for sr in SeqIO.parse('../../Data/MSU7/all.cds', 'fasta'):
    t.append(sr.id)
for m in df7['model'].to_list():
    if m not in t:
        print(m)

In [1236]:
#isolate all candidate cdses
t = []
for sr in SeqIO.parse('../../Data/MSU7/all.cds', 'fasta'):
    t.append((sr.id, sr.description, sr.seq))

In [1237]:
#write all candidate cdses
with open('../Sequence and SNP data/427 candidate cds.fasta', 'a') as f:
    for r in t:
        if r[0] in df7['model'].to_list():
            f.write('>%s\n%s\n' %(r[1], r[2]))

In [1238]:
#test
c = 0
for sr in SeqIO.parse('../Sequence and SNP data/427 candidate cds.fasta', 'fasta'):
    c+=1
print(c)

427


In [ ]:
#take a trip to beautiful intron masker

In [1294]:
t3 = []
for sr in SeqIO.parse('../Sequence and SNP data/427extracted_genes_noint.fasta', 'fasta'):
    t3.append((sr.id, sr.seq))

In [1295]:
#drop sequence duplicates
df8 = pd.DataFrame(t3).drop_duplicates(1)

In [1243]:
#write unique sequences
for i,s in df7.itertuples(index = False):
    with open('../Sequence and SNP data/392 unique candidates.fasta', 'a') as f:
        f.write('>%s\n%s\n' % (i, str(s)))

In [1244]:
#test
c = 0
for sr in SeqIO.parse('../Sequence and SNP data/392 unique candidates.fasta', 'fasta'): 
    c+=1
print(c)

392


In [1300]:
#annotate
df7 = pd.merge(df5, df7[['QTL', 'locus', 'model']], left_on = 'Gene', right_on = 'locus').drop(columns = ['locus'])

In [1302]:
#get position in gene
df7['Gene POS'] = df7['POS'] - df7['Start']

In [1303]:
#make unique identifier
t = []
for sr in SeqIO.parse('../Sequence and SNP data/392 unique candidates.fasta', 'fasta'):
    t.append((sr.id, sr.seq))
df8 = pd.DataFrame(t, columns = ['Hit key', 'Gene Seq'])

In [1304]:
df7 = df7.rename(columns = {'model' : 'Hit key'})

In [1307]:
df8 = pd.merge(df7,df8)

In [1309]:
#get reverse complements for genes from reverse strand
t = [] 
for s, sq in df8[['Strand', 'Gene Seq']].itertuples(index = False):
    if s == 'reverse':
        t.append(sq.reverse_complement())
    else:
        t.append(sq)
df8['5to3 seq'] = t

In [1327]:
#test integrity of reference alleles
t = []
for r,s,hk,snp,sqf,sqr in df8[['REF', 'Strand', 'Hit key', 'Gene POS', 'Gene Seq', '5to3 seq']].itertuples(index = False): 
    if s == 'forward':
        if not (sqf[snp:(snp + len(r))].lower() == r.lower() or '-' in sqf[snp:(snp + len(r))].lower()):
            print(sqf[snp:(snp + len(r))].lower(), r.lower(), hk)
            t.append(hk)
    else:
        if not (sqr[snp:(snp + len(r))].lower() == r.lower() or '-' in sqr[snp:(snp + len(r))].lower()):
            print(sqr[snp:(snp + len(r))].lower(), r.lower(), hk)
            t.append(hk)

g c LOC_Os01g66030.2
c a LOC_Os02g14860.1
c g LOC_Os02g14860.1
a c LOC_Os07g36600.1
t g LOC_Os07g36600.1
t g LOC_Os07g36600.1
a g LOC_Os07g36600.1
t c LOC_Os07g36600.1
a c LOC_Os07g36600.1
a c LOC_Os07g36600.1
a g LOC_Os09g15480.2
ctaa ctaat LOC_Os09g15500.1
atct ggct LOC_Os12g27994.1
ctgc ctgt LOC_Os12g27994.1
c g LOC_Os12g27994.1
c g LOC_Os12g27994.1
t a LOC_Os12g27994.1
c g LOC_Os12g27994.1
g a LOC_Os12g27994.1
c t LOC_Os12g27994.1
c g LOC_Os12g27994.1
c a LOC_Os12g27994.1
gcctggg gagcttc LOC_Os12g27994.1
c t LOC_Os12g41460.1
c a LOC_Os12g41460.1
c t LOC_Os12g41460.1
g c LOC_Os12g41460.1
g a LOC_Os12g41460.1
c g LOC_Os12g41460.1
g c LOC_Os12g41460.1
g t LOC_Os12g41460.1
g t LOC_Os12g41460.1
c g LOC_Os12g41460.1
g c LOC_Os12g41460.1
c g LOC_Os12g41460.1
c g LOC_Os12g41460.1
g c LOC_Os12g41460.1
t g LOC_Os12g41460.1
t c LOC_Os12g41460.1
c t LOC_Os12g41460.1
c t LOC_Os12g41460.1
g c LOC_Os12g41460.1
g a LOC_Os12g41460.1
g t LOC_Os12g41460.1
t c LOC_Os12g41460.1
g c LOC_Os12g41460.1
g t

In [1329]:
#remove faulty alignments
t2 = []
for a in set(t):
    t2 = t2 + list(df8[df8['Hit key'] == a].index)

In [1337]:
df9 = df8.drop(index = t2)

In [1331]:
#test integrity of reference alleles again
t = []
for r,s,hk,snp,sqf,sqr in df9[['REF', 'Strand', 'Hit key', 'Gene POS', 'Gene Seq', '5to3 seq']].itertuples(index = False): 
    if s == 'forward':
        if not (sqf[snp:(snp + len(r))].lower() == r.lower() or sqf[snp].lower() == '-'):
            print(sqf[snp:(snp + len(r))].lower(), r.lower(), hk)
            t.append(hk)
    else:
        if not (sqr[snp:(snp + len(r))].lower() == r.lower() or sqr[snp].lower() == '-'):
            print(sqr[snp:(snp + len(r))].lower(), r.lower(), hk)
            t.append(hk)

gg--- ggggt LOC_Os01g66310.1
tcagtatttgta------ tcagtatttgtactggag LOC_Os03g03520.1
t--- tggc LOC_Os05g30454.1
at- atc LOC_Os12g41660.1
gg-- ggtc LOC_Os12g41720.1


In [395]:
#deprecated. skip.
#not needed right now
#correcting reference substitutions
t = []
for r,s,hk,snp,sqf,sqr in df9[['REF', 'Strand', 'Hit key', 'Gene POS', 'Gene Seq', '5to3 seq']].itertuples(index = False): 
    if s == 'forward':
        if not (sqf[snp].lower() == r.lower() or sqf[snp].lower() == '-'):
            if (len(r) != 1):
                if not (sqf[snp:snp+len(r)].lower() == r.lower() or sqf[snp].lower() == '-'):
                    tsq = str(sqf[:snp]) + r + str(sqf[snp + len(r):])
                    t.append(Seq(tsq, generic_dna))
                else:
                    t.append(Seq(str(sqf), generic_dna))
            else:
                tsq = str(sqf[:snp]) + r + str(sqf[snp + len(r):])
                t.append(Seq(tsq, generic_dna))        
        else:
            t.append(Seq(str(sqf), generic_dna))
    else:
        if not (sqr[snp].lower() == r.lower() or sqr[snp].lower() == '-'):
            if (len(r) != 1):
                if not (sqr[snp:snp+len(r)].lower() == r.lower() or sqr[snp].lower() == '-'):
                    tsq = str(sqr[:snp]) + r + str(sqr[snp + len(r):])
                    t.append(Seq(tsq, generic_dna))
                else:
                    t.append(Seq(str(sqr), generic_dna))
            else:
                tsq = str(sqr[:snp]) + r + str(sqr[snp + len(r):])
                t.append(Seq(tsq, generic_dna))
        else:
            t.append(Seq(str(sqr), generic_dna))

In [396]:
#swapping with rename
#df9['Extract'] = t

In [1338]:
#make cds column
df9 = df9.rename(columns = {'5to3 seq' : 'Extract'}).drop(columns = ['Gene Seq'])

In [1340]:
#test alignments one more time
for r, snp, sq in df9[['REF', 'Gene POS', 'Extract']].itertuples(index = False): 
    if not (sq[snp].lower() == r.lower() or sq[snp].lower() == '-'):
        if (len(r) != 1):
            if not (sq[snp:snp+len(r)].lower() == r.lower() or sq[snp].lower() == '-'):
                print(sq[snp:snp+len(r)].lower(), r.lower())

gg--- ggggt
tcagtatttgta------ tcagtatttgtactggag
t--- tggc
at- atc
gg-- ggtc


In [1342]:
#expand multiallelic SNPs into new row
t = []
for i,r in df9.iterrows():
    if ',' in r['ALT']:
        for a in r['ALT'].split(','):
            r2 = r.copy()
            r2.loc['ALT'] = a
            t.append(r2)
    else:
        t.append(r)

In [1343]:
df10 = pd.DataFrame(t)

In [1345]:
#mutagenesis
t = []
for r, a, p, esq in df10[['REF', 'ALT', 'Gene POS', 'Extract']].itertuples(index = False):
    if esq[p] == '-':
        t.append(Seq(str(esq), generic_dna))
    else:
        tsq = str(esq)
        tsq = tsq[:p] + a + tsq[p+len(r):]
        t.append(Seq(tsq, generic_dna))

In [1346]:
df10['Mutated'] = t

In [1347]:
#check if frameshift occurred
t = []
for esq, msq in df10[['Extract', 'Mutated']].itertuples(index = False):
    if len(esq) != len(msq) and (len(msq) - len(esq)) % 3 != 0:
        t.append('yes')
    else:
        t.append('no')    

In [1348]:
df10['Frameshift'] = t

In [1349]:
#test frameshifts
for i,r in df10[df10['Frameshift'] == 'yes'].iterrows():
    if len(r['REF']) != 1 or len(r['ALT']) != 1:
        if (len(r['ALT']) - len(r['REF'])) % 3 == 0:
            print(r['REF'], r['ALT'])
        #if r['Frameshift'] == 'no':
            #print(r['REF', 'ALT'])

In [1350]:
# mark substitutions as synonymous/missense/nonsense

t = []
for p, r, a, s, e, m in df10[['Gene POS', 'REF', 'ALT', 'Strand', 'Extract', 'Mutated']].itertuples(index = False):
    
#     if len(r) < len(a):
#         t.append('insertion')
#         continue
#     elif len(r) > len(a):
#         t.append('deletion')
#         continue
    
    if s == 'forward':
    
        op = Seq(str(e).replace('-', ''), generic_dna).translate()
        np = Seq(str(m).replace('-', ''), generic_dna).translate()
        if op == np:
            t.append('synonymous')
        else:
            
            no_nonsense_switch = 'on'
            for i in range(min(len(op), len(np))):
                if np[i] != op[i]:
                    if np[i] == '*':
                        t.append('nonsense')
                        no_nonsense_switch = 'off'
                        break
            if no_nonsense_switch == 'on':
                t.append('missense')
            
            
    else:
        op = Seq(str(e).replace('-', ''), generic_dna).reverse_complement().translate()
        np = Seq(str(m).replace('-', ''), generic_dna).reverse_complement().translate()
        if op == np:
            t.append('synonymous')
        else:
            
            no_nonsense_switch = 'on'
            for i in range(min(len(op), len(np))):
                if np[i] != op[i]:
                    if np[i] == '*':
                        t.append('nonsense')
                        no_nonsense_switch = 'off'                        
                        break 
            if no_nonsense_switch == 'on':
                t.append('missense')


c:\users\michael\appdata\local\programs\python\python38\lib\site-packages\Bio\Seq.py:2738: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


In [1351]:
df10['Effect'] = t

In [1352]:
# mark site as intron/exon
t = []
for p, e in df10[['Gene POS', 'Extract']].itertuples(index = False):
    if e[p] == '-':
        t.append('intron')
    else:
        t.append('exon')

In [1353]:
df10['Site'] = t

In [1354]:
df10 = df10.reset_index(drop = True)

In [1356]:
df10['MID'] = df10.index

In [1359]:
df10.drop(columns = ['Extract', 'Mutated']).to_clipboard(index = False)
#run testcases before saving from clipboard
#rename hit key to transcript

#### `test center`

In [1382]:
str(Seq(str(df10.loc[161, 'Extract']).replace('-', ''), generic_dna).translate())

'MGEFSGMATTGALYSGQVMPITSRDGSGTKPKGTRLERAIRDLQKIAAEYRPPAIDINEVDPNGQVAVKRRLPPEVKQKLAKVARLSANHGKIQEHELMDRLMGIVGHLVQRRTLRRNMKEMVESGLSAKQEKADKFQRVKMEINEMIKSRVAAKAKVNEHHSGSADDFQIANDEKRYLKGKSVMDAALEDRICDLYDLYVEGMDEDKGPQSRKLYVELAELWPEGSMDNVGIKDAINRSKERRRSLYNQQKVRNEERMKRKRLAAAAKLQDGYPVVMQSALIQQVAQPPITNPVATYPVTDQGSKSFDRVREISASANPDDINRNTGEMKKKKRKPESDLVDTQANAMKGPSQHVEKNKPPKRADEAVETVLCLPFYDQQPS*'

In [1383]:
str(Seq(str(df10.loc[161, 'Mutated']).replace('-', ''), generic_dna).translate())

'MGEFSGMATTGALYSGQVMPITSRDGSGTKPKGTRLERAIRDLQKIAAEYRPPAIDINEVDPNGQVAVKRRLPPEVKQKLAKVARLSANHGKIQEHELMDRLMGIVGHLVQRRTLRRNMKEMVESGLSAKQEKADKFQRVKMEINEMIKSRVAAKAKVNEHHSGSADDFQIANDEKRYLKGKSVMDAALEDRICDLYDLYVEGMDEDKGPQSRKLYVELAELWPEGSMDNVGIKDAINRSKERRRSLYNQQKVRNEERMKRKRLAAAAKLQDGYPVVMQSALIQQVAQPPITNPVTDQGSKSFDRVREISASANPDDINRNTGEMKKKKRKPESDLVDTQANAMKGPSQHVEKNKPPKRADEAVETVLCLPFYDQQPS*'

In [1384]:
str(Seq(str(df10.loc[1033, 'Mutated']).replace('-', ''), generic_dna).reverse_complement().translate())

'MEKEVEEEEEEEEAEASRKKGERGLGEGECGAPSSKKRDPELELRMKQKAAEWHRKAREETLKEIAKEMAKYPNEDWSDTPGVKAREYREDWEYRWSAIFGPYDTISPIPPMRYTHRKDDSMPRHISVRHTLQIISVKIKGIRGGLQWPINVFGLIAARDTIDRNRIMIFNRTRDNCQTITKEDR*LLLTGPTRAVVVSDPVYFEAPLKVKGSVESEDKDLSFLAVPLTGASDRGETRLVNREYTSRLSTLELTFGFVVESLEASISVRIIDGSWKDGFRGAFTAHTPSLKDNKVLLLDSGYCEMVPVTADRMIKLSRHVVSVEGEGDLTVSVLALGTDNVIEDEKDFTPKEAGMSQSSLDVGFCKLEVTVNWSLLSLLPDGYT*'

In [1385]:
str(Seq(str(df10.loc[1033, 'Extract']).replace('-', ''), generic_dna).reverse_complement().translate())

'MEKEVEEEEEEEEAEASRKKGERGLGEGECGAPSSKKRDPELELRMKQKAAEWHRKAREETLKEIAKEMAKYPNEDWSDTPGVKAREYREDWEYRWSAIFGPYDTISPIPPMRYTHRKDDSMPRHISVRHTLQIISVKIKGIRGGLQWPINVFGLIAARDTIDRNRIMIFNRTRDNCQTITKEDRYLLLTGPTRAVVVSDPVYFEAPLKVKGSVESEDKDLSFLAVPLTGASDRGETRLVNREYTSRLSTLELTFGFVVESLEASISVRIIDGSWKDGFRGAFTAHTPSLKDNKVLLLDSGYCEMVPVTADRMIKLSRHVVSVEGEGDLTVSVLALGTDNVIEDEKDFTPKEAGMSQSSLDVGFCKLEVTVNWSLLSLLPDGYT*'

In [1379]:
x1 = str(Seq(str(df10.loc[160, 'Mutated']).replace('-', ''), generic_dna).reverse_complement().translate())
#str(Seq(str(df10.loc[121, 'Mutated']).replace('-', ''), generic_dna).translate())

In [1363]:
x2 = str(Seq(str(df10.loc[8, 'Extract']).replace('-', ''), generic_dna).reverse_complement().translate())

In [1364]:
for i in range(len(x1)):
    if x1[i] != x2[i]:
        print(i, x1[i], x2[i])

347 N I


In [566]:
str(Seq(str(df10.loc[69, 'Extract']).replace('-', ''), generic_dna).reverse_complement().translate()) == str(Seq(str(df10.loc[69, 'Mutated']).replace('-', ''), generic_dna).reverse_complement().translate())

False

In [1386]:
df10 = df10.drop(columns = ['Extract', 'Mutated'])

#### `pipeline passed all tests`
***

In [ ]:
#write table

In [1388]:
pd.read_csv('../Figures and tables/All candidate genes.csv')

,Locus ID,QTL,annotation,Functionally annotated,Outlier in expression,Expression cluster,Special interest
0,LOC_Os01g65920,qCDP1,"F-box/LRR-repeat protein 2, putative, expressed",Yes,No,1,No
1,LOC_Os01g66000,qCDP1,"NADH dehydrogenase I subunit N, putative, expr...",No,No,1,No
2,LOC_Os01g66020,qCDP1,"protein kinase family protein, putative, expre...",No,No,2,No
3,LOC_Os01g66030,qCDP1,OsMADS2 - MADS-box family gene with MIKCc type...,No,No,1,No
4,LOC_Os01g66040,qCDP1,"receptor kinase, putative, expressed",No,No,1,No
...,...,...,...,...,...,...,...
335,LOC_Os12g41690,qCDP17,membrane associated DUF588 domain containing p...,No,No,1,No
336,LOC_Os12g41700,qCDP17,"LSD1 zinc finger domain containing protein, ex...",Yes,No,1,No
337,LOC_Os12g41710,qCDP17,"Protein kinase domain containing protein, expr...",No,No,2,No
338,LOC_Os12g41715,qCDP17,"DEAD-box ATP-dependent RNA helicase, putative,...",No,No,1,No


In [1396]:
df10 = pd.read_excel('../Figures and tables/340 substitution effects.xlsx')
df10 = pd.merge(df10, pd.read_csv('../Figures and tables/All candidate genes.csv').drop(columns = ['QTL'])
                , left_on = 'Gene', right_on = 'Locus ID')

In [1390]:
sum(df10['Gene'] == df10['Locus ID'])

28188

In [1397]:
df10.columns

Index(['#CHROM', 'POS', 'ID', 'Frameshift', 'Effect', 'Site', 'REF', 'ALT',
       'Gene', 'Strand', 'Start', 'End', 'QTL', 'Transcript', 'Gene POS',
       'MID', 'Locus ID', 'annotation', 'Functionally annotated',
       'Outlier in expression', 'Expression cluster', 'Special interest'],
      dtype='object')

In [1398]:
df10 = df10[['ID', '#CHROM', 'POS', 'REF', 'ALT', 'Site', 'Frameshift', 'Effect', 'QTL', 'Transcript', 'Gene POS', 'Expression cluster',
     'Functionally annotated', 'Outlier in expression', 'annotation']]

In [1400]:
df10.to_csv('../Figures and tables/SNP effects annotated.csv', index = False)

In [ ]:
pd.merge(pd.read_excel('../Figures and tables/340 substitution effects.xlsx'),
    pd.read_csv('../Figures and tables/SNP effects annotated.csv')[['Transcript', 
        'Expression cluster', 'Functionally annotated', 'Outlier in expression', 'annotation']]).drop_duplicates().to_csv('../Figures and tables/340 substitution effects annotated.csv', index = False)

#### `ANOVA and t test`

In [3]:
df11 = pd.read_excel('../Figures and tables/340 substitution effects.xlsx')

In [4]:
df12 = pd.read_csv('../../WFC2.vcf', sep = '\t')

In [22]:
df11

,#CHROM,POS,ID,Frameshift,Effect,Site,REF,ALT,Gene,Strand,Start,End,QTL,Transcript,Gene POS,MID
0,chr01,38275059,1365463,no,synonymous,intron,A,G,LOC_Os01g65920,reverse,38275055,38279030,qCDP1,LOC_Os01g65920.1,4,0
1,chr01,38275062,1365464,no,synonymous,intron,G,GGTTACAGA,LOC_Os01g65920,reverse,38275055,38279030,qCDP1,LOC_Os01g65920.1,7,1
2,chr01,38275065,1365465,no,synonymous,intron,C,T,LOC_Os01g65920,reverse,38275055,38279030,qCDP1,LOC_Os01g65920.1,10,2
3,chr01,38275069,1365466,no,synonymous,intron,A,G,LOC_Os01g65920,reverse,38275055,38279030,qCDP1,LOC_Os01g65920.1,14,3
4,chr01,38275069,1365466,no,synonymous,intron,A,T,LOC_Os01g65920,reverse,38275055,38279030,qCDP1,LOC_Os01g65920.1,14,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28183,chr12,25840275,15180417,no,synonymous,exon,C,G,LOC_Os12g41720,reverse,25838564,25840479,qCDP17,LOC_Os12g41720.1,1711,28183
28184,chr12,25840309,15180418,no,missense,exon,G,C,LOC_Os12g41720,reverse,25838564,25840479,qCDP17,LOC_Os12g41720.1,1745,28184
28185,chr12,25840315,15180419,no,missense,exon,G,C,LOC_Os12g41720,reverse,25838564,25840479,qCDP17,LOC_Os12g41720.1,1751,28185
28186,chr12,25840379,15180420,no,missense,exon,C,A,LOC_Os12g41720,reverse,25838564,25840479,qCDP17,LOC_Os12g41720.1,1815,28186


In [5]:
df4 = pd.read_table('../Sequence and SNP data/Bout340can.tsv', header = None)
df4['Start'] = df4[[8,9]].min(axis = 1)
df4['End'] = df4[[8,9]].max(axis = 1)
t = []
for s,e in df4[[8,9]].itertuples(index = False):
    if s < e:
        t.append('forward')
    else:
        t.append('reverse')
df4['Strand'] = t

In [6]:
df5 = df12

In [7]:
t = []
for c, p in df5[['#CHROM', 'POS']].itertuples(index = False):
    f = False
    for l,o,s,e in df4[df4[1] == c][[0, 'Strand', 'Start', 'End']].itertuples(index = False):
        if s <= p <= e:
            t.append((l, o, s, e))
            f = True
            break
    if f == False:
        t.append(('None', o, s, e))

In [8]:
df5['Gene'] = list(zip(*t))[0]
df5['Strand'] = list(zip(*t))[1]
df5['Start'] = list(zip(*t))[2]
df5['End'] = list(zip(*t))[3]

In [9]:
df5 = df5[df5['Gene'] != 'None']

In [10]:
t = []
for i,r in df5.iterrows():
    if ',' in r['ALT']:
        for a in r['ALT'].split(','):
            r2 = r.copy()
            r2.loc['ALT'] = a
            t.append(r2)
    else:
        t.append(r)

In [11]:
df6 = pd.DataFrame(t)

In [12]:
for a in df6['ALT'].to_list():
    if ',' in a:
        print(a)

In [13]:
df6 = df6.applymap(lambda x: 0 if x == '0/0' else x).applymap(lambda x: str(x).count('/0') if '/0' in str(x) else x).applymap(lambda x: str(x).count('0/') if '0/' in str(x) else x).applymap(lambda x: 2 if '/' in str(x) else x)

In [14]:
df6

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,IRIS_313-10340,...,IRIS_313-9606,IRIS_313-9617,IRIS_313-9626,IRIS_313-9661,IRIS_313-9682,IRIS_313-9696,Gene,Strand,Start,End
0,chr01,38275059,1365463,A,G,.,PASS,.,GT,2,...,2,2,2,2,0,2,LOC_Os01g65920,reverse,38275055,38279030
1,chr01,38275062,1365464,G,GGTTACAGA,.,PASS,.,GT,0,...,0,0,0,0,0,0,LOC_Os01g65920,reverse,38275055,38279030
2,chr01,38275065,1365465,C,T,.,PASS,.,GT,2,...,2,2,2,2,0,2,LOC_Os01g65920,reverse,38275055,38279030
3,chr01,38275069,1365466,A,G,.,PASS,.,GT,2,...,2,2,2,2,0,2,LOC_Os01g65920,reverse,38275055,38279030
3,chr01,38275069,1365466,A,T,.,PASS,.,GT,2,...,2,2,2,2,0,2,LOC_Os01g65920,reverse,38275055,38279030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119596,chr12,25840275,15180417,C,G,.,PASS,.,GT,0,...,0,0,0,0,0,0,LOC_Os12g41720,reverse,25838564,25840479
119597,chr12,25840309,15180418,G,C,.,PASS,.,GT,2,...,2,2,1,2,2,2,LOC_Os12g41720,reverse,25838564,25840479
119598,chr12,25840315,15180419,G,C,.,PASS,.,GT,2,...,0,2,0,2,2,0,LOC_Os12g41720,reverse,25838564,25840479
119599,chr12,25840379,15180420,C,A,.,PASS,.,GT,0,...,0,0,0,0,0,0,LOC_Os12g41720,reverse,25838564,25840479


In [15]:
df7 = df6.drop(columns = ['#CHROM', 'POS', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'Gene', 'Strand', 'Start', 'End'])

In [47]:
df7 = pd.read_csv('../../WFC2L.vcf', sep = '\t')

In [50]:
df7 = df7.drop(columns = ['#CHROM', 'POS', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT'])

In [16]:
df8 = pd.read_table('../../../../Desktop/39ProcessedTraits.txt').transpose()

In [52]:
t = []
for i,r in df7.iterrows():
    for tr in df8.index:
        tdf = pd.concat([r[1:], df8.loc[tr, :]], axis = 1)
        g1 = tdf[tr][tdf[i] == 0].dropna()
        g2 = tdf[tr][tdf[i] == 1].dropna()
        g3 = tdf[tr][tdf[i] == 2].dropna()

        if min(len(g1), len(g2), len(g3)) < 4:
            continue

        x = stats.f_oneway(g1, g2, g3)

        t.append((r['MID'], len(g1), len(g2), len(g3), x.statistic, x.pvalue, tr))

In [53]:
df9 = pd.DataFrame(t, columns = ['MID', 'AA', 'Aa', 'aa', 'F statistic', 'p value', 'Trait'])

In [55]:
df9

,MID,AA,Aa,aa,F statistic,p value,Trait
0,33,20,4,147,1.301004,0.274987,Root_weight
1,33,20,4,147,1.049574,0.352371,Shoot_weight
2,33,20,4,147,2.544710,0.081518,Root_length
3,33,20,4,139,0.902027,0.407799,Filtered_Shoot_length
4,33,20,4,139,2.381646,0.095667,Filtered_Root_thickness
...,...,...,...,...,...,...,...
186095,28109,112,4,57,0.100022,0.904871,Height_WO_husk
186096,28109,112,4,57,0.851065,0.428771,Width_WO_husk
186097,28109,112,4,57,0.052342,0.949020,Seed_volume
186098,28109,106,4,57,0.469777,0.625980,Filtered_Seed_Weight


In [57]:
df10 = pd.read_csv('../Figures and tables/340 substitution effects annotated.csv')[['MID', 'Frameshift', 'Effect', 'Site', 
                                                                       'Transcript', 'ID', 'QTL', 'Expression cluster', 
                                                                       'Functionally annotated', 'Outlier in expression',
                                                               'annotation']]

In [58]:
df10 = pd.merge(df9, df10, how = 'left').sort_values('p value')

In [61]:
df10.drop_duplicates().to_csv('../Figures and tables/One-way ANOVA.csv', index = False)

In [63]:
t = []
for i,r in df7.iterrows():
    for tr in df8.index:
        tdf = pd.concat([r[1:], df8.loc[tr, :]], axis = 1)
        g1 = tdf[tr][tdf[i] == 0].dropna()
        g2 = tdf[tr][tdf[i] == 1].dropna()
        g3 = tdf[tr][tdf[i] == 2].dropna()

        if min(len(g1), len(g3)) < 5:
            continue

        x = stats.ttest_ind(g1, g3)

        t.append((r['MID'], len(g1), len(g3), x.statistic, x.pvalue, tr))

In [64]:
df11 = pd.DataFrame(t, columns = ['MID', 'AA', 'aa', 't statistic', 'p value', 'Trait'])

In [65]:
df12 = pd.read_csv('../Figures and tables/340 substitution effects annotated.csv')[['MID', 'Frameshift', 'Effect', 'Site', 
                                                                       'Transcript', 'ID', 'QTL', 'Expression cluster', 
                                                                       'Functionally annotated', 'Outlier in expression',
                                                               'annotation']]

In [66]:
df12 = pd.merge(df11, df12, how = 'left').sort_values('p value')

In [71]:
df12.drop_duplicates().to_csv('../Figures and tables/t-test.csv', index = False)